In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.5f}'.format
%matplotlib inline
#sns.set(style='whitegrid', palette='muted', font_scale=1.5)
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#pd.set_option('display.max_rows', 20)

In [ ]:
!pip install openpyxl

In [ ]:
data = pd.read_excel("/kaggle/input/housing-data/housingData_7R.xlsx")

In [ ]:
data = df.copy()

In [ ]:
df = data.copy()

In [ ]:
#data = df.copy()

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data = data.drop('Unnamed: 0', axis = 1)

In [ ]:
data.describe(include = "all")

In [ ]:
data['adDescription'].head()

In [ ]:
data['adDealType'].value_counts()

In [ ]:
data['Deal_Type'] = data['adDealType'].copy()

In [ ]:
target_map = {
    10: 'rental',
    20: 'sell'}

data.Deal_Type.replace(target_map, inplace = True)

In [ ]:
data['Deal_Type'].head()

In [ ]:
data.columns

In [ ]:
len(data.columns)

In [ ]:
original_col = data.columns

new_col = ['ID', 'DealType', 'Canton', 'ZipCode',
           'City','PublishedDate', 'AvailableDate', 'PriceText', 
           'Description', 'LangDetected', 'NumRooms', 'Floor', 
           'YearOfConstruction', 'NumApartments', 'Floor1', 
           'NumApartments1', 'LivingSpace', 'BuildingArea', 
           'UsefulArea', 'CoordE', 'CoordN' ,'Deal_Type']

data = data.rename(columns=dict(zip(original_col,new_col)))

data.head()

In [ ]:
# replace all column names (in place)
#new_cols = [‘column_x’, ‘column_y’, ‘column_z’]
#df.columns = new_cols

## EDA

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(8, 5))
sns.countplot(data['Deal_Type'])
plt.title('Target variable distribution')
plt.show()

In [ ]:
data.shape

In [ ]:
data.isnull().sum().sort_values(ascending = False)

In [ ]:
#Plot missing values in data
ax = data.isna().sum().sort_values().plot(kind = 'barh', figsize = (9, 10))
plt.title('Percentage of Missing Values Per Column Dataset', fontdict={'size':15})
for p in ax.patches:
    percentage ='{:,.0f}%'.format((p.get_width()/data.shape[0])*100)
    width, height =p.get_width(),p.get_height()
    x=p.get_x()+width+0.02
    y=p.get_y()+height/2
    ax.annotate(percentage,(x,y))

In [ ]:
total = data.isnull().sum().sort_values(ascending=False)
percentage = ((data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)*100)
missing_data = pd.concat([total, percentage], axis=1, keys=['Total', 'Percentage'])
missing_data.head(20)

## Treating Missing Values and Outliers

In [ ]:
data.isnull().sum().sort_values(ascending=False)

In [ ]:
mis_columns = ['NumApartments', 'NumApartments1', 'BuildingArea', 'UsefulArea' ]

In [ ]:
data.corr()

In [ ]:
data['NumApartments'].head()

In [ ]:
data['NumApartments'].value_counts()

In [ ]:
mis_columns

In [ ]:
NumApartments = data['NumApartments'].copy()
BuildingArea = data['BuildingArea'].copy()
UsefulArea = data['UsefulArea'].copy()

In [ ]:
data.columns

In [ ]:
outliers = []
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers# Driver code

#sample_outliers = detect_outliers_zscore(data['adNumApartments.1'])
#print("Outliers from Z-scores method: ", sample_outliers)

In [ ]:
sns.boxplot(data['NumApartments'])
plt.show()

In [ ]:
sns.distplot(data['NumApartments'])
plt.show()

In [ ]:
data = data.drop(columns = mis_columns, axis = 1)

In [ ]:
data.columns

In [ ]:
data.isnull().sum().any()

In [ ]:
data.isnull().sum().sort_values(ascending = False)

In [ ]:
data['City'].value_counts()

In [ ]:
City = data['City'].copy()

In [ ]:
data['City'].describe()

In [ ]:
data['City'].head()

In [ ]:
#Filling Null Values
data['City'].fillna('Zürich', inplace = True) #Fillna
#data['City'].ffill(axis = 0)  #ForwardFill

In [ ]:
data['City'].mode()

In [ ]:
data.isnull().sum().sort_values(ascending = False)

In [ ]:
# ptype_encode = {}
# ptype_encode_values = range(16,0,-1)
# for i,k in zip(type_count.index,ptype_encode_values):
#     ptype_encode[i]=k
# ptype_encode

# data['adCity'] = data['adCity'].map(ptype_encode)

#### **Exploring the City copied from data['City']**

In [ ]:
City.head()

In [ ]:
City = pd.DataFrame(data = City)
City.head()

In [ ]:
City.info()

In [ ]:
encoder=ce.TargetEncoder(cols='City')

In [ ]:
data['City'].head()

In [ ]:
data['City'].head(20)

In [ ]:
data['City'].describe()

In [ ]:
data.describe(include = 'all')

In [ ]:
#sns.scatterplot(x = data['City'], y = data['PriceText'])
#plt.show()

## Exploring the target Column

### PriceText

In [ ]:
PriceText = data['PriceText'].copy()
PriceText = pd.DataFrame(data = PriceText)
PriceText.head()

In [ ]:
data['PriceText'].head()

In [ ]:
data['PriceText'] = data['PriceText'].replace(',','', regex = True)
data['PriceText'].head()

In [ ]:
data['PriceText'] = data['PriceText'].replace('CHF','', regex = True)
data['PriceText'].head()

In [ ]:
data['PriceText'].describe()

In [ ]:
data['PriceText'].isnull().sum()

In [ ]:
data['PriceText'].value_counts()

In [ ]:
data['PriceText'] = data['PriceText'].replace('On request',np.nan)
data['PriceText'].value_counts()

In [ ]:
data['PriceText'].isnull().sum()

In [ ]:
ax = data.isna().sum().sort_values().plot(kind = 'barh', figsize = (9, 10))
plt.title('Percentage of Missing Values Per Column Dataset', fontdict={'size':15})
for p in ax.patches:
    percentage ='{:,.0f}%'.format((p.get_width()/data.shape[0])*100)
    width, height =p.get_width(),p.get_height()
    x=p.get_x()+width+0.02
    y=p.get_y()+height/2
    ax.annotate(percentage,(x,y))

In [ ]:
data['PriceText'] = data['PriceText'].replace('CHF','', regex = True)
data['PriceText'].head()

In [ ]:
data['PriceText'].value_counts()

In [ ]:
data['PriceText'] = data['PriceText'].replace('EUR','', regex = True)
data['PriceText'].head()

In [ ]:
data['PriceText'].dtype

In [ ]:
data['PriceText'].value_counts()

In [ ]:
data['PriceText'] = data['PriceText'].apply(pd.to_numeric)

In [ ]:
data['PriceText'].dtype

In [ ]:
data['PriceText'].describe()

In [ ]:
sns.distplot(data['PriceText'])

In [ ]:
sns.boxplot(data['PriceText'])

In [ ]:
outliers = []
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers# Driver code

In [ ]:
sample_outliers = detect_outliers_zscore(data['PriceText'])
print("Outliers from Z-scores method: ", sample_outliers)

Since, PriceText contains outliers, drop the rows

In [ ]:
data = data[data["PriceText"]<min(sample_outliers)].reset_index(drop = True)

In [ ]:
data['PriceText'].describe()

In [ ]:
sns.distplot(data['PriceText'])

In [ ]:
sample_outliers = detect_outliers_zscore(data['PriceText'])
#print("Outliers from Z-scores method: ", sample_outliers)

In [ ]:
data = data[data["PriceText"]<min(sample_outliers)].reset_index(drop = True)

In [ ]:
data["PriceText"].describe()

In [ ]:
sns.boxplot(data['PriceText'])

In [ ]:
plt.boxplot(data["PriceText"])
plt.title("Detecting outliers using Boxplot")
plt.xlabel('Sample')

In [ ]:
sns.distplot(data['PriceText'])

**Encoding the categorical column with **

In [ ]:
City.shape, data['PriceText'].shape, data['City'].shape

In [ ]:
City = data['City'].copy()

In [ ]:
#Fit and Transform Train Data
encoder.fit_transform(City,data['PriceText'])

In [ ]:
data.isnull().sum()

Dropping the description for now, Will work on it later

In [ ]:
Description = data['Description'].copy()

In [ ]:
data = data.drop('Description', axis = 1)

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data.shape

### Canton

In [ ]:
data['Canton'].head()

In [ ]:
data['Canton'].value_counts()

Run it at the later end because it will add to the number of columns and there is still plenty of work to do

In [ ]:
#data = pd.get_dummies(data = data['Canton'], drop_first = True)

In [363]:
obj_col = ['Canton']

### ZipCode

In [ ]:
data.ZipCode.head()

In [ ]:
data.ZipCode.value_counts()

Is this column really relevant to the model????

In [ ]:
data.columns

## Date_Columns

In [ ]:
date_col = ['PublishedDate', 'AvailableDate']

In [ ]:
data[date_col].head()

In [ ]:
data['AvailableDate'].value_counts()

In [ ]:
data['AvailableDate'].head()

In [ ]:
#sofort = 'right away'
#nach Vereinbarung = by arrangement

In [ ]:
data['PublishedDate'].head()

In [ ]:
data.columns

In [ ]:
data['Description'] = Description.copy()

In [ ]:
data[['PublishedDate', 'AvailableDate', 'Deal_Type', 'PriceText', 'Description']].head()

In [ ]:
data['PublishedDate'].head()

In [ ]:
AvlDate = data['AvailableDate'].copy()
PubDate = data['PublishedDate'].copy()

In [ ]:
#AvlDate = pd.to_datetime(AvlDate)
AvlDate

In [ ]:
AvlDate.value_counts()

In [ ]:
replacement_mapping_dict = {
    "nach Vereinbarung": np.nan,
    "sofort": np.nan,
    "By arrangement": np.nan,
    "Immediately": np.nan,
    "not available": np.nan
}
AvlDate = AvlDate.replace(replacement_mapping_dict)

In [ ]:
AvlDate.value_counts()

In [ ]:
len(data)

In [ ]:
(AvlDate.isnull().sum()/len(data))*100

In [ ]:
AvlDate.fillna("Others", inplace = True)

### Pend Pend Pend Pend

## We Move to other columns again******

In [ ]:
data.columns

In [ ]:
NumRooms = data['NumRooms'].copy()

In [ ]:
data['NumRooms'].head()

In [ ]:
def percentage_missing(df):    
    pm = (df.isnull().sum()/len(data))*100
    return pm

In [ ]:
(data.NumRooms.isnull().sum()/len(data))*100

In [ ]:
data['NumRooms'].value_counts()

In [ ]:
data['NumRooms'] = data['NumRooms'].replace('None', np.nan)
data['NumRooms'].head()

In [ ]:
data['NumRooms'] = data['NumRooms'].apply(pd.to_numeric)

In [ ]:
data['NumRooms'].isnull().sum()

In [ ]:
percentage_missing(data['NumRooms'])

In [ ]:
data['NumRooms'].value_counts()

In [ ]:
data['NumRooms'].dtype

In [ ]:
sns.boxplot(data['NumRooms'])

In [ ]:
data['NumRooms'].describe()

In [ ]:
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers# Driver code

In [ ]:
sample_outliers = detect_outliers_zscore(data['NumRooms'])
print("Outliers from Z-scores method: ", sample_outliers)

In [ ]:
data = data[data["NumRooms"]<min(sample_outliers)].reset_index(drop = True)

In [ ]:
sns.boxplot(data['NumRooms'])
plt.title("Detecting outliers using Boxplot")
plt.xlabel('Sample')

In [ ]:
data['NumRooms'].describe()

In [ ]:
sns.distplot(data['NumRooms'])

In [ ]:
data['NumRooms'].isnull().sum()

In [ ]:
#data['NumRooms'].groupby()

In [ ]:
percentage_missing(data['NumRooms'])

In [ ]:
data.columns

### Floor Column

In [ ]:
data[['Floor', 'Floor1']].head()

In [ ]:
data = data.drop('Floor1', axis = 1)

In [ ]:
Floor = data['Floor'].copy()

In [ ]:
data['Floor'].value_counts()

In [ ]:
#To remove the digits from the floor column
data['Floor'] = data['Floor'].replace('\d+', '', regex = True)

In [358]:
data['Floor'].value_counts()

Stock                 202254
not available         183951
EG                     48024
. floor                15921
Untergeschoss           1775
. Stock oder höher        31
Name: Floor, dtype: int64

* untergeschoss = basement 
* Stock oder höher = floor or higher

* This is the equivalent of this words in English

In [365]:
# The replace fucntion to replace the words in german in the english format and correct the format of the incorrect value
replacement_mapping_dict = {
    ". Stock": 'Stock',
    ". floor ": 'Floor',
    "Untergeschoss": 'basement',
    ". Stock oder höher": 'floor or higher',
    "EG": 'ground floor'
}
data['Floor'] = data['Floor'].replace(replacement_mapping_dict)

In [366]:
data['Floor'].value_counts()

Stock              202254
not available      183951
ground floor        48024
Floor               15921
basement             1775
floor or higher        31
Name: Floor, dtype: int64

In [364]:
data[['Floor', 'NumRooms']].head()

,Floor,NumRooms
0,not available,3.50000
1,Stock,3.00000
2,Stock,10.00000
3,Stock,5.50000
4,Stock,2.50000


In [367]:
data.columns

Index(['ID', 'DealType', 'Canton', 'ZipCode', 'City', 'PublishedDate',
       'AvailableDate', 'PriceText', 'LangDetected', 'NumRooms', 'Floor',
       'YearOfConstruction', 'LivingSpace', 'CoordE', 'CoordN', 'Deal_Type',
       'Description'],
      dtype='object')

## Things to try out

* **data = data.sort_values(by=['adPublishedDate']).reset_index(drop=True)**
* Trying to sort the data by dates... 'adPublishedDate', 'adAvailableDate'
* How many days is between the published date and available date
* Extract days and month from the date columns

### Year of Construction

In [368]:
data['YearOfConstruction'].head()

0    not available
1    not available
2    not available
3             2012
4             1991
Name: YearOfConstruction, dtype: object

In [371]:
data['YearOfConstruction'].value_counts()

not available    274563
2021              16651
2020              11708
2022               7657
2019               7155
                  ...  
1761                  1
1718                  1
1612                  1
4                     1
1574                  1
Name: YearOfConstruction, Length: 483, dtype: int64

In [ ]:
#Remember this for some visualization
#count_plot = sns.catplot(x="adNumApartments.1", kind="count" ,data=data, hue="DealType")
#plt.hist(data['adNumApartments.1'])
#plt.show()